In [2]:
# Imports
import random
import string
from br_nome_gen import pessoa_random
from psycopg2 import connect

In [3]:
# Criação da classe de conexão com o banco de dados
class ConnectDB:

    def __init__(self):

        self.con = connect(
            user='postgres',
            password='root123',
            host='localhost',
            port='5432',
            dbname='bd_Nubank_Cripto',
        )
        self.cur = self.con.cursor()

    def insert_row(self, data):
        """Adiciona uma nova linha na tabela.

        :param data: (tuple) Tupla contendo os dados.
        """
        query = '''INSERT INTO cliente
        (cpf_cliente, nome_cliente, data_nascimento, contato_celular, email) 
        VALUES (%s, %s, %s, %s, %s);'''
        try:
            self.cur.execute(query, data)
        except Exception as e:
            # rollback reverte/desfaz a operação.
            self.con.rollback()
            # print('\n[x] Falha ao inserir registro [x]')
            # print(f'[x] Revertendo operação (rollback) [x]: {e}\n')
        else:
            self.con.commit()
            # print('\n[!] Registro inserido com sucesso [!]')

In [4]:
# função geradora de cpf
def gerador_cpf():
    retorno = ""
    for item in range(11):
        x = random.choice(string.digits);
        retorno += x
    return retorno

In [5]:
# Função geradora de nome e emails
# a geração desses 2 campos juntas foi necessaria para facilitar a criação do 'email' com o mesmo nome randomico presente no campo 'nome'
# por isso este metodo retorna um dicionario com 2 chaves (nome, email) 
def gerador_nome_email():
    pessoa = pessoa_random()
    email = f"{pessoa.nome.split(' ')[0]}@gmail.com"
    retorno = {'nome':pessoa.nome, 'email':email}
    return retorno

In [6]:
def gerador_data_nascimento():
    ano = random.randint(1950, 2007)
    mes = random.randint(1, 12)

    if mes == 1 or mes == 3 or mes == 5 or mes == 7 or mes == 8 or mes == 10 or mes == 12:
        dia = random.randint(1, 31)
    elif mes == 4 or mes == 6 or mes == 9 or mes == 11:
        dia = random.randint(1, 30)
    elif mes == 2:
        dia = random.randint(1, 28)
    return f"{ano}-{mes}-{dia}"

In [26]:
def gerador_celular():
    inicio = '99'
    resto = ""
    for item in range(7):
        x = random.choice(string.digits);
        resto += x
    return f"{inicio}{resto}"
    

In [ ]:
# Criando a conexão com o banco.
database = ConnectDB()

QTD_REGISTROS = 1000000
for item in range (QTD_REGISTROS):
    # Criando o registro para a inserção
    nome_e_email = gerador_nome_email()
    user = (gerador_cpf(), nome_e_email['nome'], gerador_data_nascimento(), gerador_celular(), nome_e_email['email'])
    # Inserindo um registro tabela.
    database.insert_row(data=user)

# Fechando conexão com o banco.
database.con.close()